In [1]:
import pandas as pd
import numpy as np
import re

### 智联招聘数据预处理

In [120]:
class ZhilianDataPreprocess:
    def __init__(self):
        pass
    
# 删除最后一行和无用的列
    def __general(self,df):
        if 'Unnamed: 0' in df.columns:
            df_new = df.drop(['company_webpage','job_url','scrape_time','Unnamed: 0','feedback_rate','company_address'],inplace= False,axis = 1)
    #         删除最后一行
            df_new = df_new.drop(df.index[-1])
        else:
            df_new = df.drop(['company_webpage','job_url','scrape_time','feedback_rate','company_address'],inplace= False,axis = 1)
    #         删除最后一行
            df_new = df_new.drop(df.index[-1])
        return df_new

    # salary 处理
    # 去掉工资后边的字符，例如'10000-15000元/月\xa0'，保留10000-15000
    def __salary_yuchuli(self,x):
        if (x is not np.nan) and (x is not None): 
            pattern1 = r"(.+)元"
            match1 = re.findall(pattern1,x)
            if len(match1) != 0:
                return match1[0]
            else:
                return str(-1)
        else:
            return str(-1)
        
    # 得到最小工资 ，并转换为数字，该工资为年薪（月薪*15），若为空值，则标为-1   
    def __min_salary(self,x):
        pattern2 = r"(.+)-"
        match2 = re.findall(pattern2,str(x))
        if len(match2) != 0:
            return int(match2[0])*15
        elif x != str(-1):
            return int(x)*15
        else:
            return -1
        
    # 得到最高薪资，并转换为数字，该工资为年薪（月薪*15），若为空值，则标为-1  
    def __max_salary(self,x):
        pattern3 = r".-(.+)"
        match3 = re.findall(pattern3,str(x))
        if len(match3) != 0:
            return int(match3[0])*15
        elif x != str(-1):
            return int(x)*15
        else:
            return -1
        
    #  salary完整处理，输入：df，返回处理好的salary相关列
    def salary_process(self,df):
        df = self.__general(df)
        df['salary'] = df['salary'].apply(self.__salary_yuchuli).apply(lambda x :x.strip())
        df['salary_min'] = df['salary'].apply(self.__min_salary)
        df['salary_max'] = df['salary'].apply(self.__max_salary)
        df['salary_avg'] = (df['salary_min'] + df['salary_max']) / 2
        df.drop('salary',inplace = True,axis = 1)
        return df[['salary_min','salary_max','salary_avg']]
    
#     work_experience预处理，若为空值，则返回‘不限’
    def __work_experience_yuchuli(self,x):
        if (x is not np.nan) and (x is not None): 
            pattern = r"(.+)年"
            match = re.findall(pattern,x)
            if len(match) != 0:
                return match[0]
            else:
                return '不限'
        else:
            return '不限'
#         work_experience完整处理，输入df,返回job_experience的相关列
    def work_experience_process(self,df):
        df = self.__general(df)
        df['job_experience'] = df['work_experience'].apply(self.__work_experience_yuchuli)
        df.drop('work_experience',inplace = True,axis = 1)
        return df[['job_experience']]
    
#     招聘人数处理，去掉最后边的“人”字，如果是空值，则返回-1
    def __demand_yuchuli(self,x):
        if (x is not np.nan) and (x is not None): 
            pattern = r"(.+)人"
            match = re.findall(pattern,x)
            if len(match) != 0:
                return int(match[0])
            else:
                return int(x)
        else:
            return -1
#         demand_number处理，输入df，返回处理好的demand_number
    def demand_number_process(self,df):
        df = self.__general(df)
        df['demand_number'] = df['demand_number'].apply(self.__demand_yuchuli)
        return df[['demand_number']]
    
#     company_scale预处理
    def __company_scale_yuchuli(self,x):
        if (x is not np.nan) and (x is not None): 
            pattern = r"(.+)人"
            match = re.findall(pattern,x)
            if len(match)!= 0:
                return match[0]
            else:
                return -1
        else:
            return -1
        
    def company_scale_process(self,df):
        df = self.__general(df)
        df['company_scale'] = df['company_scale'].apply(self.__company_scale_yuchuli)
        return df[['company_scale']]

#     先处理时间，再处理日期，输入df，返回处理后的发布日期和发布时间
    def publish_date_process(self,df):
        df = self.__general(df)
        df['publish_time'] = df['publish_date'].apply(lambda x :str(x)[10:])
        df['publish_date'] = df['publish_date'].apply(lambda x :str(x)[:10])
        return df[['publish_date','publish_time']]
    
#     把education_degree中的'其他'改为不限
    def education_process(self,df):
        df = self.__general(df)
        df.loc[df['education_degree'] == '其他','education_degree'] = '不限'
        return df[['education_degree']]

#     把未更改的列修改一下列名
    def no_change(self,df):
        df = self.__general(df)
        df_new = df.rename(columns= {'job_name':'welfare',
                           'job_category':'job_name',
                            'company_industrial':'company_industry'})
        df_new['job_description'] = np.nan
        df_new['work_district'] = np.nan
        return df_new[['job_name','job_description','job_nature','work_position','work_district','company_name','company_industry','company_nature','welfare']]
    
    
    
    
    def data_process(self,df):
        salary = self.salary_process(df)
        job_experience = self.work_experience_process(df)
        demand_number = self.demand_number_process(df)
        company_scale = self.company_scale_process(df)
        publish = self.publish_date_process(df)
        edu = self.education_process(df)
        other = self.no_change(df)
        final = pd.concat([other,salary,job_experience,edu,demand_number,company_scale,publish,],axis=1)
        return final

In [101]:
# 将数据读入
path = './data/jobInfo_2018_06_29.csv'
zhilian = pd.read_csv(path)

In [102]:
zhilian.head()

,Unnamed: 0,company_address,company_industrial,company_name,company_nature,company_scale,company_webpage,demand_number,education_degree,feedback_rate,job_category,job_name,job_nature,job_url,publish_date,salary,scrape_time,work_experience,work_position
0,0,\r\n北京市东城区天坛东路74号北玻大厦501室,IT服务(系统/数据/维护),北京伟东科技有限公司,合资,20-99人,NaN,2人,本科,50%,网络运营管理,大数据营销专员,全职,http://jobs.zhaopin.com/CC455672880J0005870411...,2018-06-28 19:24:22,6001-8000元/月,2018_06_29_04_10_19,不限,上海
1,1,\r\n北京西城区南滨河路23号,IT服务(系统/数据/维护),北京中关新才科技有限公司,民营,100-499人,www.zgxc.cc,36人,大专,100%,软件工程师,诚聘AIX高端运维工程师实习生,全职,http://jobs.zhaopin.com/216380732285350.htm,2018-06-28 09:26:18,8001-10000元/月,2018_06_29_04_10_21,不限,天津
2,2,\r\n北京西城区南滨河路23号,IT服务(系统/数据/维护),北京中关新才科技有限公司,民营,100-499人,www.zgxc.cc,36人,大专,100%,实习生,技术支持/IT运维实习生,全职,http://jobs.zhaopin.com/216380732273926.htm,2018-06-28 09:26:57,8001-10000元/月,2018_06_29_04_10_24,不限,北京
3,3,\r\n北京西城区南滨河路23号,IT服务(系统/数据/维护),北京中关新才科技有限公司,民营,100-499人,www.zgxc.cc,36人,大专,100%,IT技术支持/维护工程师,聘AIX Linux运维实习生,全职,http://jobs.zhaopin.com/216380732279322.htm,2018-06-28 09:28:17,6000-12000元/月,2018_06_29_04_10_25,不限,北京
4,4,\r\n沙坡头区平安西路宁夏大学中卫校区云中心,IT服务(系统/数据/维护),宁夏誉成云创数据投资有限公司,国企,20-99人,NaN,1人,本科,NaN,副总裁/副总经理,副总经理分管销售,全职,http://jobs.zhaopin.com/CC421164838J0006946240...,2018-06-26 14:38:37,20000-40000元/月,2018_06_29_04_10_27,5-10年,北京


In [103]:
zl = ZhilianDataPreprocess()

In [104]:
zl.data_process(zhilian)

,job_name,job_description,job_nature,work_position,work_district,company_name,company_industry,company_nature,welfare,salary_min,salary_max,salary_avg,job_experience,education_degree,demand_number,company_scale,publish_date,publish_time
0,网络运营管理,NaN,全职,上海,NaN,北京伟东科技有限公司,IT服务(系统/数据/维护),合资,大数据营销专员,90015,120000,105007.5,不限,本科,2,20-99,2018-06-28,19:24:22
1,软件工程师,NaN,全职,天津,NaN,北京中关新才科技有限公司,IT服务(系统/数据/维护),民营,诚聘AIX高端运维工程师实习生,120015,150000,135007.5,不限,大专,36,100-499,2018-06-28,09:26:18
2,实习生,NaN,全职,北京,NaN,北京中关新才科技有限公司,IT服务(系统/数据/维护),民营,技术支持/IT运维实习生,120015,150000,135007.5,不限,大专,36,100-499,2018-06-28,09:26:57
3,IT技术支持/维护工程师,NaN,全职,北京,NaN,北京中关新才科技有限公司,IT服务(系统/数据/维护),民营,聘AIX Linux运维实习生,90000,180000,135000.0,不限,大专,36,100-499,2018-06-28,09:28:17
4,副总裁/副总经理,NaN,全职,北京,NaN,宁夏誉成云创数据投资有限公司,IT服务(系统/数据/维护),国企,副总经理分管销售,300000,600000,450000.0,5-10,本科,1,20-99,2018-06-26,14:38:37
5,软件工程师,NaN,全职,深圳,NaN,中青中关村软件人才基地,计算机软件,国企,软件工程师 Python（AI）开发实习生,112500,210000,161250.0,不限,大专,22,500-999,2018-06-28,08:15:05
6,数据库开发工程师,NaN,实习,深圳,NaN,深圳市深链云技术服务有限公司,IT服务(系统/数据/维护),民营,实习程序员,45000,60000,52500.0,不限,本科,2,20-99,2018-06-28,19:24:15
7,数据库管理员,NaN,全职,上海,NaN,远东百货,贸易/进出口,外商独资,数据库架构师,120015,150000,135007.5,1-3,本科,1,500-999,2018-06-28,13:28:08
8,Java开发工程师,NaN,全职,北京,NaN,北京智和信通技术有限公司,计算机软件,民营,高级java工程师（网络安全）,270000,375000,322500.0,3-5,本科,2,20-99,2018-06-28,15:07:10
9,用户界面（UI）设计,NaN,全职,深圳,NaN,深圳市标越科技有限公司,IT服务(系统/数据/维护),民营,ui设计师,120000,180000,150000.0,3-5,大专,1,500-999,2018-06-28,20:57:41


In [117]:
big = pd.read_json('./data/jobInfo_2018_08_04.json')
big.head()

,company_address,company_industrial,company_name,company_nature,company_scale,company_webpage,demand_number,education_degree,feedback_rate,job_category,job_name,job_nature,job_url,publish_date,salary,scrape_time,work_experience,work_position
0,\n北京昌平区,网络游戏,北京心遇科技有限公司,保密,20人以下,NaN,1人,本科,None,游戏设计/开发,unity工程师,全职,http://jobs.zhaopin.com/CZ642375830J0003619961...,2018-08-03 13:50:28,10000-20000元/月,2018_08_04_03_10_40,3-5年,北京
1,\n软件园3期诚毅大街366号,计算机软件,影像图(厦门)信息科技有限公司,民营,20-99人,www.xmyxttech.com,5人,本科,None,算法工程师,视觉工程师,全职,http://jobs.zhaopin.com/CC466975981J0012149790...,2018-08-03 15:30:11,10001-15000元/月,2018_08_04_03_10_43,3-5年,深圳
2,\n朝阳区朝阳北路225号京龙大厦5层,IT服务(系统/数据/维护),北京朗镜科技有限责任公司,民营,20-99人,www.lenztechretail.com,1人,本科,None,其他,自动化测试工程师,全职,http://jobs.zhaopin.com/CC278710388J0006559381...,2018-08-03 13:50:52,15001-20000元/月,2018_08_04_03_10_46,3-5年,北京
3,\n朝阳区朝阳北路225号京龙大厦5层,IT服务(系统/数据/维护),北京朗镜科技有限责任公司,民营,20-99人,www.lenztechretail.com,1人,本科,None,IOS开发工程师,ios开发工程师,全职,http://jobs.zhaopin.com/CC278710388J0006122101...,2018-08-03 13:50:52,10001-15000元/月,2018_08_04_03_10_50,3-5年,北京
4,\n北京市朝阳区白家庄路甲6号2幢一层1096,计算机软件,北京市软通合力信息技术有限公司,民营,20-99人,NaN,2人,本科,None,软件研发工程师,python工程师,全职,http://jobs.zhaopin.com/CC543237529J0006764740...,2018-08-03 14:02:48,8000-12000元/月,2018_08_04_03_10_54,1-3年,北京


In [121]:
b = ZhilianDataPreprocess()

In [122]:
b.company_scale_process(big)

,company_scale
0,20
1,20-99
2,20-99
3,20-99
4,20-99
5,20-99
6,20
7,100-499
8,100-499
9,100-499


In [124]:
tmp = b.data_process(big)
tmp

,job_name,job_description,job_nature,work_position,work_district,company_name,company_industry,company_nature,welfare,salary_min,salary_max,salary_avg,job_experience,education_degree,demand_number,company_scale,publish_date,publish_time
0,游戏设计/开发,NaN,全职,北京,NaN,北京心遇科技有限公司,网络游戏,保密,unity工程师,150000,300000,225000.0,3-5,本科,1,20,2018-08-03,13:50:28
1,算法工程师,NaN,全职,深圳,NaN,影像图(厦门)信息科技有限公司,计算机软件,民营,视觉工程师,150015,225000,187507.5,3-5,本科,5,20-99,2018-08-03,15:30:11
2,其他,NaN,全职,北京,NaN,北京朗镜科技有限责任公司,IT服务(系统/数据/维护),民营,自动化测试工程师,225015,300000,262507.5,3-5,本科,1,20-99,2018-08-03,13:50:52
3,IOS开发工程师,NaN,全职,北京,NaN,北京朗镜科技有限责任公司,IT服务(系统/数据/维护),民营,ios开发工程师,150015,225000,187507.5,3-5,本科,1,20-99,2018-08-03,13:50:52
4,软件研发工程师,NaN,全职,北京,NaN,北京市软通合力信息技术有限公司,计算机软件,民营,python工程师,120000,180000,150000.0,1-3,本科,2,20-99,2018-08-03,14:02:48
5,C语言开发工程师,NaN,全职,广州,NaN,广州市热马科技有限公司,通信/电信/网络设备,民营,初级C++开发工程师/可实习,52500,90000,71250.0,不限,不限,4,20-99,2018-08-03,14:37:10
6,互联网软件工程师,NaN,全职,北京,NaN,北京华兴宏视技术发展有限公司,IT服务(系统/数据/维护),民营,web后端工程师,105000,180000,142500.0,不限,本科,2,20,2018-07-20,15:37:52
7,游戏设计/开发,NaN,全职,成都,NaN,四川魔力风暴科技有限公司,网络游戏,民营,动漫游戏CG专员应届毕业生/学徒,60015,90000,75007.5,不限,不限,5,100-499,2018-08-03,18:35:25
8,WEB前端开发,NaN,全职,北京,NaN,盘锦辽河数码科技发展有限公司,IT服务(系统/数据/维护),股份制企业,前端开发工程师,150015,225000,187507.5,1-3,本科,2,100-499,2018-08-03,19:03:27
9,软件工程师,NaN,全职,广州,NaN,广州艾科网络科技有限公司,互联网/电子商务,民营,初级java开发工程师,60000,75000,67500.0,1,大专,5,100-499,2018-08-03,16:24:15


In [130]:
tmp['company_scale'].value_counts()

100-499      18442
20-99        12479
500-999       6901
1000-9999     6635
10000         1080
20            1075
-1              98
Name: company_scale, dtype: int64